## Construct Numpy Dataset from TF datasets

In [15]:
# Construct lists of file paths and corresponding prefixes
import os

sequence_dir = "/Volumes/JOE_LOUIS/peaks.bedgraph/bed_cleaned/sequences"

dataset_files = []
# List all files in directory
for file_name in os.listdir(sequence_dir):
    
    # helping to store file paths and prefix into final list
    temp = []

    # Full Path to file
    file_path = os.path.join(sequence_dir,file_name)
    
    # Generating the prefix name
    prefix_name = file_name.split('.')[1]

    # storing file paths and prefixes into dataset files
    temp.extend([file_path,prefix_name])
    dataset_files.append(temp)

In [16]:
# creating a list of dataframes containing sequence information
import pandas as pd
import numpy as np

dataset_files = pd.DataFrame(dataset_files,columns=['file_path','file_prefix'])
datasets_out = []

for i in dataset_files.index: # use for loop when running through all files
    
    # store file name
    file_path = dataset_files.loc[i,"file_path"]
    file_name = dataset_files.loc[i,"file_prefix"]
    
    # updating the user
    print(f"Processing {i} out of {dataset_files.index[-1]} files...",end="\r")
    
    # Storing sequence and relevant information
    sequence_info = []
    sequence_labels = []

    # read file 
    with open(file_path,'r') as file:
        
        # Extracting usful information regarding the file
        lines = file.readlines()
        for line in lines:
            if line.startswith('>'):
                line = line.rstrip().lstrip('>')
                line = line.split(':')
                line = [item for i in line for item in i.split('-')]
                
                if line[0] == "chrX":
                    x_status = True
                else:
                    x_status = False
                
                line.append(x_status)
                line.insert(0,file_name)

                sequence_labels.append(line)
            else:
                line = line.rstrip().upper()
                seq_len = len(line)
                sequence_info.append([seq_len,line])
        
        # organizing file information
        sequence_labels = pd.DataFrame(sequence_labels)
        sequence_info = pd.DataFrame(sequence_info)
        sequence_out = pd.concat([sequence_labels,sequence_info],axis=1,ignore_index=True)
        sequence_out.columns = ['name','chr','start','end','Xchr','seq_len','seq']
        
        # appending into three dimension numpy file
        datasets_out.append(sequence_out)

print()
print("Done!")

Processing 74 out of 74 files...
Done!


In [17]:
# Saving the processed datasets
import pickle

with open('processed_datasets.pkl','wb') as f:
    pickle.dump(datasets_out,f)
    f.close()

In [1]:
# Loading the process datasets
import pickle

with open('processed_datasets.pkl','rb') as f:
    processed_datasets = pickle.load(f)
    f.close()

In [2]:
processed_datasets[0].head()

,name,chr,start,end,Xchr,seq_len,seq
0,Adelman_Dmel_S2_H3K27ac_ChIP-seq_rep1and2,chr2L,5474,6924,False,1450,TAAGCTCGAACATAGAACATAGGCTTGAACATATAATGACTGCCTT...
1,Adelman_Dmel_S2_H3K27ac_ChIP-seq_rep1and2,chr2L,7024,7349,False,325,ACCTATTTGCGCATATGCGTTTATTTTTGGGATTTAATTTTAACAT...
2,Adelman_Dmel_S2_H3K27ac_ChIP-seq_rep1and2,chr2L,7399,10074,False,2675,TGTAGGTGATTTTATTTATTAGAATACGAATTCTTTATCTGAATCG...
3,Adelman_Dmel_S2_H3K27ac_ChIP-seq_rep1and2,chr2L,10124,11599,False,1475,ACTTGAAGGCTCATTAACTTACTTCTCATATTGACATATTTTCTTC...
4,Adelman_Dmel_S2_H3K27ac_ChIP-seq_rep1and2,chr2L,11699,16099,False,4400,AAAACATTTAAATGTAGATACGTACAAAACAGCAAATTAAAATAGG...


In [7]:
# tokenizing the DNA sequence
import textwrap
import numpy as np

# You must process datasets 20 at a time, then merge the lists afterwards.
for index,sequence_out in enumerate(processed_datasets[71:]):
    
    # clearing space: memory issue 
    seqs_processed = []
    seq_labels_out = []
    
    # screening for sequences 1000bp and under
    seqs = sequence_out[sequence_out['seq_len'] <= 1000]['seq']
    seqs = seqs.reset_index(drop=True)

    # saving the seq labels
    labels = sequence_out[sequence_out['seq_len'] <= 1000]['Xchr']
    labels = list(labels.reset_index(drop=True))
    seq_labels_out.extend(labels)

    # saving classification id
    seq_id = sequence_out['name'][0]

    # TOKEN SIZE!!!
    token_size = 3

    for i in seqs.index:
        print(f'Processing Sample {index+71} "{seq_id}"... {i} of {seqs.index[-1]} sequences.',end='\r')

        # creating and applying pad
        seq_len = len(seqs[i])
        pad_length = 1000 - seq_len
        pad = 'N' * pad_length
        seq_temp = seqs[i] + pad

        # tokenizing
        # seq_list = textwrap.wrap(seq_temp,token_size)
        seq_list = []
        for i in range(len(seq_temp) - 2):
            seq_list.append(seq_temp[i:i+token_size])
        
        if len(seq_list[-1]) != token_size:
            seq_list = seq_list[:-1]

        # adding classification id & finish id
        seq_list.insert(0,seq_id)
        seq_list.append("<END>")

        # append to final list
        seqs_processed.append(seq_list)
    
    # exporting numpy files
    seqs_processed = np.array(seqs_processed)
    seq_labels_out = np.array(seq_labels_out) 
    np.save(f'X.{index+71}.{seq_id}.npy',seqs_processed)
    np.save(f'y.{index+71}.{seq_id}.npy',seq_labels_out)

    print()

Processing Sample 71 "GSM892323_L3mbt"... 372 of 372 sequences.
Processing Sample 72 "GSM948712_e2-4hr_FAIRE_peaks"... 12533 of 12533 sequences.
Processing Sample 73 "GSM948713_e6-8hr_FAIRE_peaks"... 10665 of 10665 sequences.
Processing Sample 74 "GSM948714_e16-18hr_FAIRE_peaks"... 13216 of 13216 sequences.


## Trial Run of Processed data with simple Neural Network

In [25]:
# obtaining keys to use in dictionary =
import numpy as np

def get_unique_items(nested_list):
    unique_items = set()
    for item in nested_list:
        if isinstance(item, list):
            unique_items.update(get_unique_items(item))
        else:
            unique_items.add(item)
    return unique_items

all_keys = get_unique_items(seqs_processed)
print(all_keys)

{'ACT', 'GSE37864_mle', 'CAN', 'GTA', 'AAN', 'CGC', 'ACC', 'CCA', 'NNN', 'NAG', 'GTG', 'CGN', 'GGA', 'ATN', 'ATG', 'GCG', 'GCN', 'TGA', 'GTN', 'TAG', 'AGC', 'CGG', 'GCC', 'GTT', 'GSE37864_msl3', 'NCT', 'Adelman_Dmel_S2_H3K4me1_ChIP-seq_rep1and2', 'ATA', 'CTN', 'TGG', 'TCC', 'NCA', 'TGC', 'GGG', 'GNN', 'CAG', 'GSE101554_S2_dRing_ChIP_peaks', 'NNG', 'GSE99004_macs_1WTgA', 'NGA', 'CAC', 'TCN', 'GTC', 'TTA', 'GSE37864_mof', 'NAA', 'NCN', 'GAA', 'GCA', 'ACA', 'CTC', 'Adelman_Dmel_S2_RNAPolII_ChIP-seq_control_rep1and2_normalized', 'GSE101554_S2_HA_ChIP', 'NGG', 'NNA', 'Adelman_Dmel_S2_H3K4me3_ChIP-seq_rep1and2', 'GAC', 'NCG', 'NTC', 'CTT', 'AAC', 'GSE101554_S2_Rpb3_ChIP_peaks', 'CNN', 'NNT', 'GAT', 'TCT', 'NGC', 'ATC', 'AGG', 'GSE37864_msl1', 'CTA', 'TCG', 'TNN', 'AGN', 'CCG', 'GAN', 'GSE101365_idr-macs2-mock_ago2_mueller', 'ATT', 'TTC', 'CCN', 'TAA', 'AGT', 'CGA', 'NAC', 'GSE99004_macs_1dH1gA', '<END>', 'Adelman_Dmel_S2_NELF-B_ChIP-seq_rep1and2', 'ACG', 'NGT', 'GGC', 'GGN', 'GSE101554_S2_M1

In [26]:
# constructing dictionary
dna_dict = {}
counter = 0 
for i in all_keys:
    dna_dict[i] = counter
    counter+=1

print("Number of Items in Dictionary:",len(dna_dict))

Number of Items in Dictionary: 129


In [ ]:
# replacing keys with values in the dataset
print("Original Sequence:",seqs_processed[0][:5])

def replace_values(nested_list, dictionary):
    """
    Given a nested list, replaces every item with the corresponding value in a dictionary.
    """
    if isinstance(nested_list, list):
        return [replace_values(item, dictionary) for item in nested_list]
    elif nested_list in dictionary:
        return dictionary[nested_list]
    else:
        return nested_list

seqs_out = replace_values(seqs_processed,dna_dict)
print("Updated Sequence:",seqs_out[0][:5])

In [82]:
# check on the labels
print(np.array(seqs_out).shape)
print(np.array(seq_labels_out).shape)

seqs_out = np.array(seqs_out)
seq_labels_out = np.array(seq_labels_out)

(69211, 1000)
(69211,)


In [ ]:
# saving sample numpy arrays
np.save('sample.X.npy',seqs_out)
np.save('sample.y.npy',seq_labels_out)

### Simple Neural Network

In [98]:
baseline = seq_labels_out[seq_labels_out == False].shape[0] / seq_labels_out.shape[0]
baseline = "{:.3f}".format(baseline)
print("Baseline Acc:",baseline)

Baseline Acc: 0.739


In [104]:
# building deep nueral network as a sanity check
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LeakyReLU,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Splitting and Scaling data
X_train,X_other,y_train,y_other = train_test_split(seqs_out,seq_labels_out,train_size=0.7,stratify=seq_labels_out,random_state=22)
X_val,X_test,y_val,y_test = train_test_split(X_other,y_other,train_size=0.5,stratify=y_other,random_state=22)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Deep Neural Network
model = Sequential([
    Dense(units=512),
    LeakyReLU(),
    Dropout(rate=0.5),
    Dense(units=1,activation='sigmoid')
])

model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = BinaryCrossentropy(),
    metrics=['acc']
)

model.fit(
    X_train,y_train,
    epochs=10,batch_size=256,
    validation_data=(X_val,y_val)
)

Epoch 1/10
190/190 [==============================] - 2s 9ms/step - loss: 0.6491 - acc: 0.6972 - val_loss: 0.5482 - val_acc: 0.7369
Epoch 2/10
190/190 [==============================] - 2s 9ms/step - loss: 0.5470 - acc: 0.7297 - val_loss: 0.5346 - val_acc: 0.7410
Epoch 3/10
190/190 [==============================] - 2s 9ms/step - loss: 0.5308 - acc: 0.7366 - val_loss: 0.5380 - val_acc: 0.7359
Epoch 4/10
190/190 [==============================] - 2s 9ms/step - loss: 0.5258 - acc: 0.7392 - val_loss: 0.5367 - val_acc: 0.7377
Epoch 5/10
190/190 [==============================] - 2s 8ms/step - loss: 0.5233 - acc: 0.7413 - val_loss: 0.5339 - val_acc: 0.7424
Epoch 6/10
190/190 [==============================] - 2s 9ms/step - loss: 0.5188 - acc: 0.7430 - val_loss: 0.5344 - val_acc: 0.7339
Epoch 7/10
190/190 [==============================] - 2s 10ms/step - loss: 0.5180 - acc: 0.7437 - val_loss: 0.5423 - val_acc: 0.7339
Epoch 8/10
190/190 [==============================] - 2s 9ms/step - loss: 0

In [102]:
model.evaluate(
    X_test,y_test,batch_size=256
)

41/41 [==============================] - 0s 3ms/step - loss: 0.5442 - acc: 0.7294


[0.5442226529121399, 0.7294355630874634]